# Threshold Tools 

 A single channel of an image is selected for either binary thresholding or auto thresholding (Gaussian, mean, Otsu, or triangle). For a color image, selecting a channel of an image for thresholding likely involves conversion from RGB to HSV or LAB color space, then selecting Hue, Saturation, Value, Lightness, Green-Magenta, or Blue-Yellow channels. It's best to select a channel that maximizes contrast between the target object and the background. When thresholding an image to segment a target object, it may not be possible to isolate just the target object. Multiple thresholding steps on various channels may be necessary as well as downstream noise reduction steps. For an example of this approach see the [VIS tutorial](vis_tutorial.ipynb).

In [ ]:
from plantcv import plantcv as pcv

In [ ]:
lass options:
    def __init__(self):
        self.image = "img/tutorial_images/vis/original_image.jpg"
        self.debug = "plot"
        self.writeimg= False
        self.outdir = "."
        
# Get options
args = options()

# Set debug to the global parameter 
pcv.params.debug = args.debug

# Read image
img, path, filename = pcv.readimage(args.image)

# Each of the threshold methods take grayscale image data and there are a few 
# different ways to transform a color image into a different color space. 
# Examples of using `rgb2gray_lab` and `rgb2gray_hsv` can be found in 

Each of the threshold methods take grayscale image data, and there are a few different ways to transform a color image into a gray color space. Examples of `rgb2gray_lab` and `rgb2gray_hsv` (and the way that multiple color spaces can be combined to further reduce noise) can be found in the [VIS tutorial](vis_tutorial.ipynb) and the [VIS/NIR tutorial](vis_nir_tutorial.ipynb). 

In [ ]:
# Convert the RGB img to grayscale 

# Inputs:
#   rgb_img - RGB image data 
gray_img = pcv.rgb2gray(img)

In [ ]:
# Create a binary image based on a threshold value 

# Inputs:
#   gray_img - Grayscale image data 
#   threhold - Threshold value (0-255), cutoff point for thresholding 
#   max_value - Value to apply above the threshold (255 = white)
#   object_type - 'light' (default) or 'dark', if the object is lighter than the 
#                 background then standard threholding is done, but if darker than
#                 background then inverse thresholding is done. 
binary_thresh1 = pcv.threshold.binary(gray_img, 30, 255, 'light')
binary_thresh2 = pcv.threshold.binary(gray_img, 60, 255, 'light')

In [ ]:
# Create a binary image using the Gaussian adaptive thresholding method

# Inputs:
#   gray_img - Grayscale image data 
#   max_value - Value to apply above threshold (255 = white)
#   object_type - 'light' (default) or 'dark', if the object is lighter than the 
#                 background then standard threholding is done, but if darker than
#                 background then inverse thresholding is done. 
gauss_thresh = pcv.threshold.gaussian(gray_img, 255, 'light')

In [ ]:
# Create a binary image using the mean adaptive thresholding method 

# Inputs:
#   gray_img - Grayscale image data 
#   max_value - Value to apply above threshold (255 = white)
#   object_type - 'light' (default) or 'dark', if the object is lighter than the 
#                 background then standard threholding is done, but if darker than
#                 background then inverse thresholding is done. 
mean_thresh = pcv.threshold.mean(gray_img, 255, 'light')

In [ ]:
# Create a binary image using Otsu's method 

# Inputs:
#   gray_img - Grayscale image data 
#   max_value - Value to apply above threshold (255 = white)
#   object_type - 'light' (default) or 'dark', if the object is lighter than the 
#                 background then standard threholding is done, but if darker than
#                 background then inverse thresholding is done. 
otsu_thresh = pcv.threshold.otsu(gray_img, 255, 'light')

In [ ]:
# Create a binary image using adaptive thresholding

# Inputs:
#   gray_img - Grayscale image data 
#   max_value - Value to apply above threshold (255 = white)
#   object_type - 'light' (default) or 'dark', if the object is lighter than the 
#                 background then standard threholding is done, but if darker than
#                 background then inverse thresholding is done. 
#   xstep - Value to move along the x-axis to determine the points from which to 
#           calculate distance (recommended to start at 1, the default, and change 
#           if needed) 
triangle_thresh = pcv.threshold.triangle(gray_img, 255, 'light', xstep=1)